In [6]:
import requests
import pandas as pd

In [7]:
df = pd.read_csv('./journal_abbreviations_acs.csv',delimiter=';',header=None)
df.rename(columns={0: "long_name", 1: "short_name"},inplace=True)
df.head()

,long_name,short_name
0,ACS Applied Materials & Interfaces,ACS Appl. Mater. Interfaces
1,ACS Applied Nano Materials,ACS Appl. Nano Mater.
2,ACS Biomaterials Science & Engineering,ACS Biomater. Sci. Eng.
3,ACS Catalysis,ACS Catal.
4,ACS Central Science,ACS Cent. Sci.


In [8]:
# doi = '10.1021/acsami.8b15684'
# doi = '10.1021/acsami.8b04600'
doi = '10.1016/j.jpcs.2020.109840'

In [9]:
bare_url = "http://api.crossref.org/"

In [10]:
def get_json(doi):
    """
    Parameters
    ----------
        doi: str
    Returns
    -------
        found: bool
        item: dict
            Response from crossref
    """

    url = "{}works/{}"
    url = url.format(bare_url, doi)
    r = requests.get(url)
    found = False if r.status_code != 200 else True
    item = r.json()

    return found, item


In [26]:
found, bib = get_bib(doi)

In [32]:
bib

"@article{Monz_n_Gonz_lez_2021,\n\tdoi = {10.1016/j.jpcs.2020.109840},\n\turl = {https://doi.org/10.1016%2Fj.jpcs.2020.109840},\n\tyear = 2021,\n\tmonth = {mar},\n\tpublisher = {Elsevier {BV}},\n\tvolume = {150},\n\tpages = {109840},\n\tauthor = {C{\\'{e}}sar R. Monz{\\'{o}}n-Gonz{\\'{a}}lez and Mar{\\'{\\i}}a Elena S{\\'{a}}nchez-Vergara and Wilmer E. Vallejo Narv{\\'{a}}ez and Tom{\\'{a}}s Rocha-Rinza and Marcos Hern{\\'{a}}ndez and Elizabeth G{\\'{o}}mez and Omar Jim{\\'{e}}nez-Sandoval and Cecilio {\\'{A}}lvarez-Toledano},\n\ttitle = {Synthesis and characterization of organotin({IV}) semiconductors and their applications in optoelectronics},\n\tjournal = {Journal of Physics and Chemistry of Solids}\n}"

In [11]:
found, js = get_json(doi)

In [46]:
def synthesize_md_string(json):
#     title
    t = json['message']['title'][0]
    title = f'- {t}\n'
#     Abstract
    if 'abstract' in json['message'].keys():
        abst = json['message']['abstract'][0]
        abstract = f'- Abstract:\n\t- {abst}'
    else:
        abstract = f'- Abstract:\n\t- \n'
    
#     References - Authors
    ref_base = '- Reference:\n'
    auth_base = '\t- Authors:\n'
    
    author_list = json['message']['author']
    a_list = []
    for index,a in enumerate(author_list):
        if index == 0:
            filename_author = a['family']
        given = a['given']
        family = a['family']
        item = f'[[{given} {family}]]'
        auth_base += '\t\t- '+item +'\n'

    ref = ref_base + auth_base 
#     Reference - Journal
    journal = json['message']['container-title']
    abbr = df[df['long_name'] == journal[0]]['short_name'].values[0]
    filename_journal = abbr.replace(' ','').replace('.','')

    journal = f'\t- Journal:\n\t\t- [[{journal[0]}]] [[{abbr}]]\n'
#     Url
    if 'url' in js['message'].keys():
        u = json['message']['url']
    elif 'URL' in js['message'].keys():
        u = json['message']['URL']
    url = f'\t- URL:\n\t\t- {u}\n'
    
    empty_fields = '- Notes:\n- Summary:\n- Concepts:\n- Keywords:\n'
    
    filename_year = json['message']['created']['date-parts'][0][0]
    
    filename = f'{filename_author}EtAl_{filename_journal}{filename_year}'
    
    entry = title + abstract + ref + journal + url + empty_fields
    
    print(filename)
    print(entry)
    
    return filename, entry
    

In [47]:
tsts = synthesize_md_string(js)

Monzón-GonzálezEtAl_JPhysChemSolids2020
- Synthesis and characterization of organotin(IV) semiconductors and their applications in optoelectronics
- Abstract:
	- 
- Reference:
	- Authors:
		- [[César R. Monzón-González]]
		- [[María Elena Sánchez-Vergara]]
		- [[Wilmer E. Vallejo Narváez]]
		- [[Tomás Rocha-Rinza]]
		- [[Marcos Hernández]]
		- [[Elizabeth Gómez]]
		- [[Omar Jiménez-Sandoval]]
		- [[Cecilio Álvarez-Toledano]]
	- Journal:
		- [[Journal of Physics and Chemistry of Solids]] [[J. Phys. Chem. Solids]]
	- URL:
		- http://dx.doi.org/10.1016/j.jpcs.2020.109840
- Notes:
- Summary:
- Concepts:
- Keywords:



In [41]:
tsts[1]

'- Synthesis and characterization of organotin(IV) semiconductors and their applications in optoelectronics\n- Abstract:\n\t- \n- Reference:\n\t- Authors:\n\t\t- [[César R. Monzón-González]]\n\t\t- [[María Elena Sánchez-Vergara]]\n\t\t- [[Wilmer E. Vallejo Narváez]]\n\t\t- [[Tomás Rocha-Rinza]]\n\t\t- [[Marcos Hernández]]\n\t\t- [[Elizabeth Gómez]]\n\t\t- [[Omar Jiménez-Sandoval]]\n\t\t- [[Cecilio Álvarez-Toledano]]\n\t- Journal:\n\t\t- [[Journal of Physics and Chemistry of Solids]] [[J. Phys. Chem. Solids]]\n\t- URL:\n\t\t- http://dx.doi.org/10.1016/j.jpcs.2020.109840\n- Notes:\n- Summary:\n- Concepts:\n- Keywords:\n'

In [43]:
with open(f'./{tsts[0]}.md','w') as fout:
    fout.write(tsts[1])

In [119]:
with open('./pa.md', 'r') as handler:
    ff = handler.read()